| Imię Nazwisko | Kierunek | Data |
| :-: | :-: | :-: |
| Andrzej Szafarczyk | Informatyka Techniczna |  29.03.2021 |


### Rozkład LU występuje, kiedy macierz A można wyrazić jako iloczyn macierzy trójkątnej dolnej L i trójkątnej górnej U:
A=LU

Rozwiązanie układu równań Ax=b dzieli się wtedy na dwa etapy:


• Rozwiązywanie: Lz=b , względem z

• Rozwiązywanie: Ux=z , względem x

Rozwiązanie tego typu układów jest bardzo proste przy zastosowaniu algorytmów:

• Podstawiania w przód dla macierzy L

• Podstawiania wstecz dla macierzy U

Trzeba zaznaczyć, że rozkład LU nie dla każdej macierzy istnieje.



### Algorytm rozkładu Cholesky’ego (rozkład Banachiewicza) macierzy na czynniki LL
T. Taki rozkład jest możliwy, gdy macierz A jest:

• Rzeczywista

• Symetryczna

• Dodatnio określona

Występuje wtedy rozkład na czynniki A=L LT taki, że L jest macierzą dolną trójkątną o elementach dodatnich na głównej przekątnej.

### Zad1 Forward i Backward sub

Forward sub

In [127]:
import numpy as np
import math
import scipy.linalg as la
a = np.array(
    [
        [6,    0,    0,    4], 
        [12,    -8,    0,    10],
        [3,    -13,    9,    3],
        [-6,    4,    1,    -18]
     ], float)
b = np.array([ 12,34,27,-38 ], float)
Acpy = np.copy(a)
Bcpy = np.copy(b)
n = len(b)
result = np.zeros(n, float)
remember = np.arange(0,n,1 )

In [128]:
def forwardSub():
    for k in range(n - 1):
        for i in range(k + 1, n):
            if a[k,k] == 0: continue
            mnoznik = a[i, k] / a[k, k]
            for j in range(k, n):
                a[i, j] -= a[k, j]  * mnoznik
                b[i] -= b[k] * mnoznik
    
forwardSub()
print(a)


[[  6.     0.     0.     4.  ]
 [  0.    -8.     0.     2.  ]
 [  0.     0.     9.    -2.25]
 [  0.     0.     0.   -12.75]]


Backward sub

In [129]:
        a=Acpy
        def backwardSub():
            size= n
            if a[size-1,size-1] == 0 : result[size-1] = 0
            else : result[size-1] = b[size-1] / a[size-1,size-1]

            for i in range(size-2,-1,-1): # going from last row up
                sumX= 0
                for j in range(i+1,n):
                    sumX += a[i,j] * result[j]

                if a[i,i] == 0:
                    result[i] = 0
                else:
                    result[i] = (b[i] - sumX) / a[i,i]
        
        backwardSub()    
        print(a)
   

[[  6.   0.   0.   4.]
 [ 12.  -8.   0.  10.]
 [  3. -13.   9.   3.]
 [ -6.   4.   1. -18.]]


# 2.Algorytm dootlit'a - dokompozycja LU 

Podstawowym problemem numerycznym w tej metodzie jest dokonanie rozkładu LU macierzy współczynników. Żeby ten rozkład macierzy był jednoznaczny, zakłada się, że elementy na głównej przekątnej jednej z macierzy, L albo U, są równe 1.

Rozkład LU jest wyznaczany za pomocą metody Doolittle’a.

Ta metoda nie jest niezawodna, tzn. podczas obliczeń może wystąpić dzielenie przez zero.

 W metodzie tej równość LU traktuje się jako układ n^2n^2 niewiadomymi. Te niewiadome to elementy ij dla i>j (elementy poniżej przekątnej), oraz ij dla j>=i (elementy na i powyżej przekątnej), przy założeniu, że na diagonali macierzy L znajdują się 1



In [130]:
A=np.array( [ [1.0,4.0,1.0],[1.0,6.0,-1.0],[2.0,-1.0,-2.0] ] )
B= np.array( [ [3,0,1],[0,-1,3,],[1,3,0] ] )
C=np.array([ [4,1/2,1],[1/2,17/16,1/4],[1,1/4,33/64] ])


def luDecomposition(mat):
    n= len(mat)
    lower = np.zeros((n,n,))
    upper = np.zeros((n,n))
 
    # Decomposing matrix into Upper and Lower triangular matrix
    for i in range(n):
        # Upper Triangular
        for k in range(i, n):
            sum = 0
            for j in range(i):
                sum += (lower[i][j] * upper[j][k])
 
            upper[i][k] = mat[i][k] - sum
 
        # Lower Triangular
        for k in range(i, n):
            if (i == k):
                lower[i][i] = 1  # Diagonal as 1
            else:
 
                # Summation of L(k, j) * U(j, i)
                sum = 0
                for j in range(i):
                    sum += (lower[k][j] * upper[j][i])
 
                # Evaluating L(k, i)
                lower[k][i] = int((mat[k][i] - sum)/upper[i][i])
    return (lower,upper)

dic = ['A','B','C']

for x,label in zip([A,B,C],dic):
    print(label)
    lower,upper = luDecomposition(x)
    print ('Lower:\n',lower,'\nUpper:\n',upper,'\n')



A
Lower:
 [[ 1.  0.  0.]
 [ 1.  1.  0.]
 [ 2. -4.  1.]] 
Upper:
 [[  1.   4.   1.]
 [  0.   2.  -2.]
 [  0.   0. -12.]] 

B
Lower:
 [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0. -3.  1.]] 
Upper:
 [[ 3.  0.  1.]
 [ 0. -1.  3.]
 [ 0.  0.  9.]] 

C
Lower:
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] 
Upper:
 [[4.       0.5      1.      ]
 [0.       1.0625   0.25    ]
 [0.       0.       0.515625]] 



 # Algorytm rozkładu Choleskiego-Banachiewicza
Rozkład Choleskiego lub rozkład Banachiewicza jest procedurą rozkładu symetrycznej, dodatnio określonej macierzy A na iloczyn postaci:

A=LL^T
gdzie L jest dolną macierzą trójkątną, a L^T jej transpozycją.

Macierz dowolnego typu można rozłożyć na iloczyn dolnej i górnej macierzy trójkątnej postaci A=LU stosując metodę LU. Jedynie w przypadku macierzy symetrycznych i dodatnio określonych możliwy jest rozkład Choleskiego. Jeśli A jest dodatnio określoną macierzą hermitowską to rozkład Choleskiego ma postać:

A=LL*

Podobnie jak rozkład LU, rozkład Choleskiego stosuje się w rozwiązywaniu równań liniowych. Stosuje się go również przy generowaniu wektorów losowych o wielowymiarowym rozkładzie normalnym.

Aby zastosować rozkład Choleskiego do rozwiązywania układów równań z niesymetryczną macierzą główną układu należy pomnożyć lewostronnie układ równań przez transpozycję macierzy głównej układu

### Algorytm choleskiego

In [131]:
A = np.array( [ [1,3],[-5,16] ] )
B = np.array( [ [4,1/2,1],[1/2,17/16,1/4,],[11,1/4,33/64] ])
C = np.array([[6, 3, 4, 8], [3, 6, 5, 1], [4, 5, 10, 7], [8, 1, 7, 25]])

def choleski(A):
    n = len(A)
    L = np.zeros( (n,n) )

    # Cholesky decomposition
    for i in range(n):
        for k in range(i+1):
            tmp_sum = sum(L[i][j] * L[k][j] for j in range(k))
            
            if (i == k): # Diagonal elements
                if( (A[i][i] - tmp_sum)<0):
                    print("Diaganalne elementy muszą być większe od zera")
                    return None
                else:   
                    L[i][k] = math.sqrt(A[i][i] - tmp_sum)
            else:
                L[i][k] = (1.0 / L[k][k] * (A[i][k] - tmp_sum))
    return L


Lb = choleski(B)
La = choleski(A)
Lc = choleski(C)

print('Macierz A po algorytmie choleski:\n',La,'\n')
print('Macierz B po algorytmie choleski:\n',Lb,'\n')
print('Macierz C po algorytmie choleski:\n',Lc,'\n')


Diaganalne elementy muszą być większe od zera
Diaganalne elementy muszą być większe od zera
Macierz A po algorytmie choleski:
 None 

Macierz B po algorytmie choleski:
 None 

Macierz C po algorytmie choleski:
 [[ 2.44948974  0.          0.          0.        ]
 [ 1.22474487  2.12132034  0.          0.        ]
 [ 1.63299316  1.41421356  2.30940108  0.        ]
 [ 3.26598632 -1.41421356  1.58771324  3.13249102]] 



#### Sprawdzenie czy A=L*LT 

In [132]:

if La != None:
    Lta= La@La.T # Transformacja otrzymanej macierzy i przemnożenie jej przez macierz z algorytmu chleskiego
    print('Sprawdzenie czy macierz jest równa macierzy pierwotnej\n',Lta,'\n')
    print('Macierz pierwotna A\n',A)


if Lb != None:
    Ltb= Lb@Lb.T
    print('\nSprawdzenie czy macierz jest równa mcierzy pierwotnej\n',Ltb,'\n')
    print('Macierz pierwotna B \n',B)
if Lc.any() != None:
    Ltc= Lc@Lc.T
    print('\nSprawdzenie czy macierz jest równa macierzy pierwotnej\n',Ltc,'\n')
    print('Macierz pierwotna C \n',C)


Sprawdzenie czy macierz jest równa macierzy pierwotnej
 [[ 6.  3.  4.  8.]
 [ 3.  6.  5.  1.]
 [ 4.  5. 10.  7.]
 [ 8.  1.  7. 25.]] 

Macierz pierwotna C 
 [[ 6  3  4  8]
 [ 3  6  5  1]
 [ 4  5 10  7]
 [ 8  1  7 25]]


# 4. Znaleźć rozkład LU i LLT, dla następującej macierzy:

In [133]:
AA=np.array([ 
             [136.01,90.86,0.0,0.0],
             [90.86,98.81,-67.59,0.0],
             [0.0,-67.59,132.01,46.26],
             [0.0,0.0,46.26,177.17]
])


### 1.Rozkład LU

In [134]:


lower,upper = luDecomposition(np.copy(AA))
print ('Lower:\n',lower,'\nUpper:\n',upper,'\n')

A=lower@upper
print("Lower * upper=\n",A)




Lower:
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]] 
Upper:
 [[136.01  90.86   0.     0.  ]
 [  0.    98.81 -67.59   0.  ]
 [  0.     0.   132.01  46.26]
 [  0.     0.     0.   177.17]] 

Lower * upper=
 [[136.01  90.86   0.     0.  ]
 [  0.    98.81 -67.59   0.  ]
 [  0.     0.   132.01  46.26]
 [  0.     0.     0.   177.17]]


Jak widać algorytm poprawnie rozłożył macierz na macierz trójkątną górną i dolną,a ich wynik mnożenia jest równy macierzy pierwotnej.

### Rozkład LLT

In [135]:
Lc = choleski(AA)

print('Macierz A po algorytmie choleski:\n',Lc,'\n')
Ltc= Lc@Lc.T
print('\nSprawdzenie czy macierz jest równa macierzy pierwotnej\n',Ltc,'\n')
print('Macierz pierwotna C \n',AA)


Macierz A po algorytmie choleski:
 [[ 11.66233253   0.           0.           0.        ]
 [  7.7908943    6.17348897   0.           0.        ]
 [  0.         -10.94842809   3.48452614   0.        ]
 [  0.           0.          13.27583668   0.96029183]] 


Sprawdzenie czy macierz jest równa macierzy pierwotnej
 [[136.01  90.86   0.     0.  ]
 [ 90.86  98.81 -67.59   0.  ]
 [  0.   -67.59 132.01  46.26]
 [  0.     0.    46.26 177.17]] 

Macierz pierwotna C 
 [[136.01  90.86   0.     0.  ]
 [ 90.86  98.81 -67.59   0.  ]
 [  0.   -67.59 132.01  46.26]
 [  0.     0.    46.26 177.17]]


Jak widać, otrzymano tę samą macierz

### 5.Algorytm Gaussa Crouta został zawarty w poprzednim sprawozdaniu